## 載入模型

In [298]:
label_to_coordinates = {
    "1-1": (0, 0), "1-2": (0.6, 0), "1-3": (1.2, 0), "1-4": (1.8, 0), "1-5": (2.4, 0), "1-6": (3.0, 0),"1-7": (3.6, 0), "1-8": (4.2, 0), "1-9": (4.8, 0), "1-10": (5.4, 0), "1-11": (6.0, 0),
    "2-1": (0, 0.6), "2-11": (6.0, 0.6),
    "3-1": (0, 1.2), "3-11": (6.0, 1.2),
    "4-1": (0, 1.8), "4-11": (6.0, 1.8),
    "5-1": (0, 2.4), "5-11": (6.0, 2.4),
    "6-1": (0, 3.0), "6-2": (0.6, 3.0), "6-3": (1.2, 3.0), "6-4": (1.8, 3.0), "6-5": (2.4, 3.0),"6-6": (3.0, 3.0), "6-7": (3.6, 3.0), "6-8": (4.2, 3.0), "6-9": (4.8, 3.0), "6-10": (5.4, 3.0), "6-11": (6.0, 3.0),
    "7-1": (0, 3.6), "7-11": (6.0, 3.6),
    "8-1": (0, 4.2), "8-11": (6.0, 4.2),
    "9-1": (0, 4.8), "9-11": (6.0, 4.8),
    "10-1": (0, 5.4), "10-11": (6.0, 5.4),
    "11-1": (0, 6.0), "11-2": (0.6, 6.0), "11-3": (1.2, 6.0), "11-4": (1.8, 6.0), "11-5": (2.4, 6.0),"11-6": (3.0, 6.0), "11-7": (3.6, 6.0), "11-8": (4.2, 6.0), "11-9": (4.8, 6.0), "11-10": (5.4, 6.0), "11-11": (6.0, 6.0)
}
label_mapping = {
    '11': '1-1','10': '1-2','9': '1-3','8': '1-4','7': '1-5','6': '1-6','5': '1-7','4': '1-8','3': '1-9','2': '1-10','1': '1-11',
    '12': '2-1','30': '2-11',
    '13': '3-1','29': '3-11',
    '14': '4-1','28': '4-11',
    '15': '5-1','27': '5-11',
    '16': '6-1','17': '6-2','18': '6-3','19': '6-4','20': '6-5','21': '6-6','22': '6-7','23': '6-8','24': '6-9','25': '6-10','26': '6-11',
    '49': '7-1','31': '7-11',
    '48': '8-1','32': '8-11',
    '47': '9-1','33': '9-11',
    '46': '10-1','34': '10-11',
    '45': '11-1','44': '11-2','43': '11-3','42': '11-4','41': '11-5','40': '11-6','39': '11-7','38': '11-8','37': '11-9','36': '11-10','35': '11-11'
}

In [299]:
from tensorflow.keras.models import load_model
from tensorflow import keras
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib  # 用於保存模型
import json
from sklearn.model_selection import train_test_split

In [300]:
# 載入模型與標準化器
base_model = load_model('Model_SAVE basedate/4 mc AP RSSI FTM StdDev 2024-12-14 data train/DNN_best_model4mcAP.h5')


scaler = joblib.load('Model_SAVE basedate/4 mc AP RSSI FTM StdDev 2024-12-14 data train/scaler_4mcAP.pkl')

/home/mcslab/anaconda3/envs/yang_cuda/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [301]:
# 凍結所有層
for layer in base_model.layers[:-1]:  # 除了最後一層 (Output Layer)
    layer.trainable = False

# 確認哪些層可訓練
base_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 64)                832       
                                                                 
 dense_45 (Dense)            (None, 128)               8320      
                                                                 
 dense_46 (Dense)            (None, 64)                8256      
                                                                 
 dense_47 (Dense)            (None, 49)                3185      
                                                                 
Total params: 20593 (80.44 KB)
Trainable params: 3185 (12.44 KB)
Non-trainable params: 17408 (68.00 KB)
_________________________________________________________________


In [302]:
base_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### 加入新資料 & 資料前處理

In [303]:
selected_columns = ['Label',
                    'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',
                                'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']  

X_testing_selected_columns = [
        'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
        'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',
                                'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']  

label_column = 'Label'
target_column = 'Label'
#  'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi'
# 'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
# 'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',

In [304]:
# 讀取測試資料 2024_12_21   2024_12_27   2025_01_03   2025_01_10   2025_02_28
test_file_path = "timestamp_allignment_Balanced_2024_12_21_rtt_logs.csv"  # 測試資料的檔案名稱
date_test = "2024_12_21"
modelname = "DNN 4 mcAPs BEST"
test_data = pd.read_csv(test_file_path, usecols=selected_columns)
test_data

,Label,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,1-11,444.0,6841.0,7353.0,5116.0,-60.0,-71.0,-50.0,-62.0,809.0,705.0,210.0,996.0
1,1-11,366.0,6841.0,7236.0,5242.0,-59.0,-71.0,-54.0,-66.0,808.0,766.0,719.0,420.0
2,1-11,288.0,8569.0,7217.0,4738.0,-58.0,-71.0,-56.0,-64.0,506.0,531.0,131.0,2157.0
3,1-11,366.0,6958.0,7327.0,6854.0,-59.0,-71.0,-52.0,-58.0,842.0,680.0,348.0,1181.0
4,1-11,327.0,6899.0,7275.0,6887.0,-59.0,-71.0,-54.0,-62.0,907.0,706.0,451.0,741.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18860,7-1,6284.0,2358.0,7783.0,594.0,-73.0,-74.0,-58.0,-58.0,849.0,991.0,933.0,576.0
18861,7-1,6372.0,2280.0,7627.0,516.0,-66.0,-75.0,-60.0,-59.0,1409.0,909.0,735.0,679.0
18862,7-1,6430.0,2202.0,7705.0,360.0,-73.0,-76.0,-60.0,-59.0,869.0,924.0,731.0,781.0
18863,7-1,6313.0,2280.0,7705.0,516.0,-73.0,-74.0,-60.0,-59.0,866.0,1198.0,750.0,598.0


In [305]:
# 資料前處理 (一): 刪除前後n筆資料
n = 10
# 確保依據Label排序
test_data = test_data.sort_values(by=label_column).reset_index(drop=True)

# 建立一個空的 DataFrame 用於存放處理後的資料
test_processed_data = pd.DataFrame(columns=test_data.columns)

# 針對每個Label群組進行處理
for label, group in test_data.groupby(label_column):
    # 刪除前n筆和後n筆資料
    if len(group) > 2 * n:  # 確保群組資料足夠
        group = group.iloc[n:-n]
    else:
        group = pd.DataFrame()  # 若資料不足，刪除整個群組
    # 將處理後的群組資料加入
    test_processed_data = pd.concat([test_processed_data, group], ignore_index=True)

# test_processed_data
# Calculate the number of rows with NaN values
nan_rows = test_processed_data.isnull().any(axis=1).sum()

# Print the result
print(f"Number of rows with NaN values: {nan_rows}")

# 找出包含 NaN 的列
rows_with_nan = test_processed_data[test_processed_data.isnull().any(axis=1)]

# # 印出這些列
# print("Rows with NaN values:")
# print(rows_with_nan)
test_data_imputed = test_processed_data.groupby(label_column).apply(
    lambda group: group.fillna(group.mean())
).reset_index()

# Calculate the number of rows with NaN values
nan_rows = test_data_imputed.isnull().any(axis=1).sum()

# Print the result
print(f"Number of rows with NaN values: {nan_rows}")

# 找出包含 NaN 的列
rows_with_nan = test_data_imputed[test_data_imputed.isnull().any(axis=1)]

test_data_imputed

Number of rows with NaN values: 1112
Number of rows with NaN values: 0


/tmp/ipykernel_1913547/195304898.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  test_processed_data = pd.concat([test_processed_data, group], ignore_index=True)


,Label,level_1,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,1-1,0,3960.0,6265.0,13105.0,-342.0,-63.0,-74.0,-65.0,-49.0,1544.0,741.0,949.0,1243.0
1,1-1,1,3999.0,6694.0,13457.0,-381.0,-63.0,-74.0,-67.0,-52.0,1511.0,508.0,1696.0,1297.0
2,1-1,2,4038.0,6460.0,13281.0,-264.0,-64.0,-75.0,-69.0,-52.0,1408.0,546.0,999.0,1244.0
3,1-1,3,4194.0,6421.0,13960.0,-381.0,-63.0,-74.0,-63.0,-48.0,1005.0,620.0,948.0,1466.0
4,1-1,4,4189.0,6460.0,13667.0,-381.0,-58.0,-73.0,-63.0,-54.0,995.0,553.0,860.0,1565.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17880,9-11,17880,1655.0,8569.0,2549.0,4825.0,-55.0,-67.0,-52.0,-67.0,354.0,70.0,606.0,491.0
17881,9-11,17881,1889.0,8452.0,2549.0,4998.0,-46.0,-67.0,-51.0,-70.0,509.0,131.0,493.0,86.0
17882,9-11,17882,1928.0,8803.0,2588.0,4998.0,-55.0,-68.0,-52.0,-71.0,215.0,211.0,444.0,87.0
17883,9-11,17883,1968.0,8481.0,2588.0,5086.0,-55.0,-67.0,-51.0,-70.0,242.0,168.0,460.0,125.0


In [306]:
reverse_label_mapping = {v: int(k) - 1 for k, v in label_mapping.items()}  # 讓數字標籤 -1

In [307]:
# 建立 Label 映射
y_test = test_data_imputed[target_column]
y_test_numeric = y_test.map(reverse_label_mapping)

print("Final reverse_label_mapping in DNN:", reverse_label_mapping)
print("y_numeric unique values in DNN:", y_test_numeric.unique())

y_test_numeric

Final reverse_label_mapping in DNN: {'1-1': 10, '1-2': 9, '1-3': 8, '1-4': 7, '1-5': 6, '1-6': 5, '1-7': 4, '1-8': 3, '1-9': 2, '1-10': 1, '1-11': 0, '2-1': 11, '2-11': 29, '3-1': 12, '3-11': 28, '4-1': 13, '4-11': 27, '5-1': 14, '5-11': 26, '6-1': 15, '6-2': 16, '6-3': 17, '6-4': 18, '6-5': 19, '6-6': 20, '6-7': 21, '6-8': 22, '6-9': 23, '6-10': 24, '6-11': 25, '7-1': 48, '7-11': 30, '8-1': 47, '8-11': 31, '9-1': 46, '9-11': 32, '10-1': 45, '10-11': 33, '11-1': 44, '11-2': 43, '11-3': 42, '11-4': 41, '11-5': 40, '11-6': 39, '11-7': 38, '11-8': 37, '11-9': 36, '11-10': 35, '11-11': 34}
y_numeric unique values in DNN: [10  1  0  9  8  7  6  5  4  3  2 45 33 44 35 34 43 42 41 40 39 38 37 36
 11 29 12 28 13 27 14 26 15 24 25 16 17 18 19 20 21 22 23 48 30 47 31 46
 32]


0        10
1        10
2        10
3        10
4        10
         ..
17880    32
17881    32
17882    32
17883    32
17884    32
Name: Label, Length: 17885, dtype: int64

In [308]:
# 把label部分拿掉
X_test = test_data_imputed.drop(columns=['level_1','Label'])

# 確保測試資料的特徵與訓練資料的特徵一致
X_test = X_test[X_testing_selected_columns]  # 選取相同的特徵

print(type(X_test))

<class 'pandas.core.frame.DataFrame'>


In [309]:
# 使用之前訓練時的標準化器 (scaler) 來標準化測試數據
X_test_scaled = scaler.transform(X_test)

## 重新訓練model

### train validation 切

In [310]:
# 會發現如果用 train_test_split的方法會有資料分布不平均問題，解決辦法如下
from sklearn.model_selection import StratifiedShuffleSplit

# 設定每個類別的固定樣本數
N_train = 1 # 訓練集每個類別至少要有 N_train 筆資料
N_val = 0
test_val_ratio = 1  # 剩餘資料中，50% 作為驗證集，50% 作為測試集

# 轉為 DataFrame 方便操作
data = pd.DataFrame(X_test_scaled)
data['label'] = y_test_numeric  # 加入 label 欄位

print((data['label'] == 10).sum())  # 直接計算 True 的數量

365


In [311]:
# 轉為 DataFrame 方便操作
data = pd.DataFrame(X_test_scaled)
data['label'] = y_test_numeric  # 加入 label 欄位

# data

In [312]:
# train_data_full = data.groupby('label', group_keys=False).apply(lambda x: x.sample(n=min(N_train, len(x)), replace=False, random_state=42)).reset_index(drop=True)
train_data_full = data.groupby('label', group_keys=False).sample(n=N_train, replace=False, random_state=42)

# 確保 `train_data_full` 內部的 `label` 數量正確
print(train_data_full['label'].value_counts())  # 每個類別應該剛好 1 筆
# train_data_full

label
0     1
25    1
27    1
28    1
29    1
30    1
31    1
32    1
33    1
34    1
35    1
36    1
37    1
38    1
39    1
40    1
41    1
42    1
43    1
44    1
45    1
46    1
47    1
26    1
24    1
1     1
23    1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    1
21    1
22    1
48    1
Name: count, dtype: int64


In [313]:
# 若 N_train 太小，則 validation set 直接從 test set 取一部分
if N_val > 0:
    # sss = StratifiedShuffleSplit(n_splits=1, test_size=N_val / len(remaining_data), random_state=42)
    # val_index, test_index = next(sss.split(remaining_data.drop(columns=['label']), remaining_data['label']))
    # val_data = remaining_data.iloc[val_index]
    # remaining_data = remaining_data.iloc[test_index]
    # **第二步**：從 `train_data_full` 裡抽 `N_val = 1` 作為 `validation set`
    sss = StratifiedShuffleSplit(n_splits=1, test_size=N_val / N_train, random_state=42)
    train_index, val_index = next(sss.split(train_data_full.drop(columns=['label']), train_data_full['label']))
    train_data = train_data_full.iloc[train_index]
    val_data = train_data_full.iloc[val_index]
    
else:
    val_data = pd.DataFrame(columns=data.columns)  # 若沒有 validation data，建立空 DataFrame
    train_data = train_data_full


In [314]:
# 剩下的資料（未被抽入 train_data_full 的部分）直接作為 test set
remaining_data = data.drop(train_data.index)
print(len(remaining_data))

17836


In [315]:
# **轉換為 NumPy 陣列**
X_train, y_train = train_data.drop(columns=['label']).values, train_data['label'].values
X_val, y_val = val_data.drop(columns=['label']).values, val_data['label'].values
X_test, y_test = remaining_data.drop(columns=['label']).values, remaining_data['label'].values

# **確認數據切分結果**
print(f"Training set: {len(X_train)} samples, {len(np.unique(y_train))} unique labels")
print(f"Validation set: {len(X_val)} samples, {len(np.unique(y_val))} unique labels")
print(f"Test set: {len(X_test)} samples, {len(np.unique(y_test))} unique labels")

Training set: 49 samples, 49 unique labels
Validation set: 0 samples, 0 unique labels
Test set: 17836 samples, 49 unique labels


In [316]:
# **計算每個 Set 內各 Label 的資料數量**
train_label_counts = pd.Series(y_train).value_counts().sort_index()
val_label_counts = pd.Series(y_val).value_counts().sort_index()
test_label_counts = pd.Series(y_test).value_counts().sort_index()

# **確保所有 Labels 都有出現在三個 Set 裡**
all_labels = sorted(set(train_label_counts.index) | set(val_label_counts.index) | set(test_label_counts.index))
label_distribution = pd.DataFrame(index=all_labels)

label_distribution["Training Set"] = train_label_counts
label_distribution["Validation Set"] = val_label_counts
label_distribution["Test Set"] = test_label_counts

In [317]:
# **用 0 填補缺失值（表示該 Label 在該 Set 中沒有數據）**
label_distribution = label_distribution.fillna(0).astype(int)

from IPython.display import display
display(label_distribution)

,Training Set,Validation Set,Test Set
0,1,0,364
1,1,0,364
2,1,0,364
3,1,0,364
4,1,0,364
5,1,0,364
6,1,0,364
7,1,0,364
8,1,0,364
9,1,0,364


## 訓練

In [318]:
import time
# 記錄開始時間
start_time = time.time()

In [319]:
from tensorflow.keras.callbacks import EarlyStopping

if N_val > 0:
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
else:
    early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)


In [320]:
# 確保變數命名一致
X_train_small = X_train  # 確保這裡用的變數和前面一致
y_train_small = y_train

In [321]:
# 設定 batch_size
batch_size = min(32, max(8, len(X_train_small) // 2))  # 避免 batch size 過大
print(batch_size)

24


In [322]:
if N_val > 0:
    base_model.fit(X_train_small, y_train_small,validation_data=(X_val, y_val), epochs=10000, batch_size=batch_size, callbacks=[early_stop])
else:
    base_model.fit(X_train_small, y_train_small, epochs=10000, batch_size=batch_size, callbacks=[early_stop])

Epoch 1/10000
3/3 [==============================] - 0s 1ms/step - loss: 6.5369 - accuracy: 0.4286
Epoch 2/10000
3/3 [==============================] - 0s 1ms/step - loss: 6.2335 - accuracy: 0.4490
Epoch 3/10000
3/3 [==============================] - 0s 881us/step - loss: 5.9997 - accuracy: 0.4898
Epoch 4/10000
3/3 [==============================] - 0s 2ms/step - loss: 5.7910 - accuracy: 0.4898
Epoch 5/10000
3/3 [==============================] - 0s 2ms/step - loss: 5.5858 - accuracy: 0.4898
Epoch 6/10000
3/3 [==============================] - 0s 2ms/step - loss: 5.4019 - accuracy: 0.4694
Epoch 7/10000
3/3 [==============================] - 0s 2ms/step - loss: 5.2209 - accuracy: 0.4694
Epoch 8/10000
3/3 [==============================] - 0s 3ms/step - loss: 5.0496 - accuracy: 0.4898
Epoch 9/10000
3/3 [==============================] - 0s 5ms/step - loss: 4.8931 - accuracy: 0.5102
Epoch 10/10000
3/3 [==============================] - 0s 3ms/step - loss: 4.7377 - accuracy: 0.5102
Epoch 1

In [323]:
# 記錄結束時間
end_time = time.time()

# 計算訓練時間（秒）
training_time = end_time - start_time
print(f"訓練時間：{training_time:.2f} 秒")

訓練時間：2.93 秒


In [324]:
print(len(X_test))

17836


In [325]:
loss, accuracy = base_model.evaluate(X_test, y_test)
print(f"Evaluation on 90% unused data - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

558/558 [==============================] - 1s 833us/step - loss: 1.2117 - accuracy: 0.8054
Evaluation on 90% unused data - Loss: 1.2117, Accuracy: 0.8054


In [326]:
# ### 預測
# base_model.evaluate(X_test_scaled, y_test_numeric)  # 確保模型在測試模式

In [327]:
# 預測測試資料
y_test_pred_numeric = base_model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_test_pred_numeric, axis=1)

# 轉換為原本的 Label
y_test_pred_labels = [label_mapping[str(num + 1)] for num in y_pred_classes]  # 補回 +1
y_test_pred_labels

559/559 [==============================] - 0s 789us/step


['1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-3',
 '1-3',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-3',
 '1-3',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-3',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '2-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-3',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-3',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',


In [328]:
# 讀取測試資料的實際 Label
y_test_actual = test_data_imputed[target_column]
test_data_imputed

,Label,level_1,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,1-1,0,3960.0,6265.0,13105.0,-342.0,-63.0,-74.0,-65.0,-49.0,1544.0,741.0,949.0,1243.0
1,1-1,1,3999.0,6694.0,13457.0,-381.0,-63.0,-74.0,-67.0,-52.0,1511.0,508.0,1696.0,1297.0
2,1-1,2,4038.0,6460.0,13281.0,-264.0,-64.0,-75.0,-69.0,-52.0,1408.0,546.0,999.0,1244.0
3,1-1,3,4194.0,6421.0,13960.0,-381.0,-63.0,-74.0,-63.0,-48.0,1005.0,620.0,948.0,1466.0
4,1-1,4,4189.0,6460.0,13667.0,-381.0,-58.0,-73.0,-63.0,-54.0,995.0,553.0,860.0,1565.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17880,9-11,17880,1655.0,8569.0,2549.0,4825.0,-55.0,-67.0,-52.0,-67.0,354.0,70.0,606.0,491.0
17881,9-11,17881,1889.0,8452.0,2549.0,4998.0,-46.0,-67.0,-51.0,-70.0,509.0,131.0,493.0,86.0
17882,9-11,17882,1928.0,8803.0,2588.0,4998.0,-55.0,-68.0,-52.0,-71.0,215.0,211.0,444.0,87.0
17883,9-11,17883,1968.0,8481.0,2588.0,5086.0,-55.0,-67.0,-51.0,-70.0,242.0,168.0,460.0,125.0


In [329]:
# 取得預測與實際座標
y_test_pred_coordinates = np.array([label_to_coordinates[label] for label in y_test_pred_labels])
y_test_actual_coordinates = np.array([label_to_coordinates[label] for label in y_test_actual])

# 計算 MDE (Mean Distance Error)
distances = np.linalg.norm(y_test_pred_coordinates - y_test_actual_coordinates, axis=1)
mean_mde = np.mean(distances)

# 記錄每個 RP 的 MDE
mde_report_test = {}
for true_label, distance in zip(y_test_actual, distances):
    if true_label not in mde_report_test:
        mde_report_test[true_label] = []
    mde_report_test[true_label].append(distance)

# 計算測試資料的 MDE 平均值
mde_report_test_avg = {label: {"mde": np.mean(dists), "count": len(dists)} for label, dists in mde_report_test.items()}

# 儲存 MDE 結果到 JSON 檔案
test_file_path = f"{modelname}_{date_test}.json"
with open(test_file_path, "w") as f:
    json.dump(mde_report_test_avg, f, indent=4)

print(f"Test Data MDE report saved to: {test_file_path}")
print(f"\nTest Data Mean MDE: {mean_mde:.4f} meters")


Test Data MDE report saved to: DNN 4 mcAPs BEST_2024_12_21.json

Test Data Mean MDE: 0.2847 meters


In [330]:
# base_model.save(f"DNN_best_model4mcAP_4week_to_5week_1dataPerRP.h5")

### test validation 切

In [331]:
# # 會發現如果用 train_test_split的方法會有資料分布不平均問題，解決辦法如下
# from sklearn.model_selection import StratifiedShuffleSplit

# # 設定每個類別的固定樣本數
# N_train = 5 # 訓練集每個類別至少要有 N_train 筆資料
# N_val = 2
# test_val_ratio = 1  # 剩餘資料中，50% 作為驗證集，50% 作為測試集

# # 轉為 DataFrame 方便操作
# data = pd.DataFrame(X_test_scaled)
# data['label'] = y_test_numeric  # 加入 label 欄位

# print((data['label'] == 10).sum())  # 直接計算 True 的數量


In [332]:
# data

In [333]:
# # 儲存訓練集
# train_data = data.groupby('label', group_keys=False).sample(n=N_train, replace=False, random_state=42)
# # print(train_data.index)  # 查看 train_data 的索引

# print((train_data['label'] == 10).sum())  # 計算 label 為 10 的數量


In [334]:
# remaining_data = data.drop(train_data.index)
# print(remaining_data['label'].value_counts().get(10, 0))  # 應該是 348


In [335]:
# # 取得剩餘資料的 features 和 labels
# X_remaining = remaining_data.drop(columns=['label'])
# y_remaining = remaining_data['label']

# print(y_remaining.value_counts().get(10, 0))

In [336]:
# # 使用 StratifiedShuffleSplit 來確保驗證集與測試集的類別比例一致
# sss = StratifiedShuffleSplit(n_splits=1, test_size=test_val_ratio, random_state=42)
# sss = StratifiedShuffleSplit(n_splits=1, test_size=N_val/N_train, random_state=42)
# for val_index, test_index in sss.split(X_remaining, y_remaining):
#     X_val, X_test = X_remaining.iloc[val_index], X_remaining.iloc[test_index]
#     y_val, y_test = y_remaining.iloc[val_index], y_remaining.iloc[test_index]

# # 轉換回 NumPy 陣列
# X_train_small, y_train_small = train_data.drop(columns=['label']).values, train_data['label'].values
# X_val, y_val = X_val.values, y_val.values
# X_test, y_test = X_test.values, y_test.values

# # 確認切分後的數據數量
# print(f"Training set: {len(X_train_small)} samples, {len(np.unique(y_train_small))} unique labels")
# print(f"Validation set: {len(X_val)} samples, {len(np.unique(y_val))} unique labels")
# print(f"Test set: {len(X_test)} samples, {len(np.unique(y_test))} unique labels")

In [337]:
# # 取出 10% 資料作為訓練資料，剩下的 90% 作為測試資料
# X_train_small, X_unused, y_train_small, y_unused = train_test_split(
#     X_test_scaled, y_test_numeric, test_size=0.99, random_state=42, stratify=y_test_numeric
# )
# # 再切出一半作為validation set and test set
# X_val, X_test, y_val, y_test = train_test_split(
#     X_unused, y_unused, test_size=0.5, random_state=42, stratify=y_unused
# )

In [338]:
# # 檢查各組資料比例
# print(f"Training set: {len(X_train_small)} samples")
# print(f"Validation set: {len(X_val)} samples")
# print(f"Test set: {len(X_test)} samples")

In [339]:
# import pandas as pd

# # 計算每個 Set 內各 Label 的資料數量
# train_label_counts = pd.Series(y_train_small).value_counts().sort_index()
# val_label_counts = pd.Series(y_val).value_counts().sort_index()
# test_label_counts = pd.Series(y_test).value_counts().sort_index()

# # # 印出統計結果
# # print("Training Set - Label Counts:")
# # print(train_label_counts)

# # print("\nValidation Set - Label Counts:")
# # print(val_label_counts)

# # print("\nTest Set - Label Counts:")
# # print(test_label_counts)

# # 確保所有 Labels 都有出現在三個 Set 裡
# all_labels = sorted(set(train_label_counts.index) | set(val_label_counts.index) | set(test_label_counts.index))
# label_distribution = pd.DataFrame(index=all_labels)

# label_distribution["Training Set"] = train_label_counts
# label_distribution["Validation Set"] = val_label_counts
# label_distribution["Test Set"] = test_label_counts

# # 用 0 填補缺失值（表示該 Label 在該 Set 中沒有數據）
# label_distribution = label_distribution.fillna(0).astype(int)

# from IPython.display import display
# display(label_distribution)
